From: https://github.com/Quorumetrix/Blender_scripts/blob/main/Mesh%20Decimation%20Pipeline.ipynb

In [2]:
import pandas as pd
import numpy as np
import os
import time
import h5py
import pyvista as pv

# Set up paths and parameters
obj_path = 'data/c3_cell_meshes/'
dec_prcnt = 95
dec_path = 'data/c3_cell_meshes/dec/'

# Load astrocyte cellids into a list
astro_list = [5101516913]

def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as f:
        vertices = np.array(f['vertices'])
        faces = np.array(f['faces'])
    return vertices, faces

def create_pyvista_mesh(vertices, faces):
    # PyVista expects faces to be in the format (n, v1, v2, v3, ...)
    faces_with_size = np.hstack([np.full((faces.shape[0], 1), faces.shape[1]), faces])
    return pv.PolyData(vertices, faces_with_size)

def process_obj(cellid, dec_prcnt, obj_path, dec_path):
    start_time = time.time()
    
    # Create the full path to the file
    file_path = os.path.join(obj_path, str(cellid) + '_0.h5')
    print(f"Loading {file_path}")
    
    # Load the file
    vertices, faces = read_h5_file(file_path)
    load_time = time.time()
    print(f"Loaded {file_path} in {load_time - start_time:.2f} seconds")
    
    # Create PyVista mesh
    mesh = create_pyvista_mesh(vertices, faces)
    print(f"Original mesh info: {mesh}")
    
    # Perform decimation
    print(f"Decimating {file_path} by {dec_prcnt}%")
    simplified_mesh = mesh.decimate(target_reduction=dec_prcnt / 100)  # Reduce to x% of the original mesh
    decimate_time = time.time()
    print(f"Decimated {file_path} in {decimate_time - load_time:.2f} seconds")
    
    # Print simplified mesh information
    print(f"Simplified mesh info: {simplified_mesh}")
    
    # Ensure the output directory exists
    if not os.path.exists(dec_path):
        os.makedirs(dec_path)
    
    # Save the simplified mesh in PLY format
    output_filename = os.path.basename(file_path).replace('.h5', '.ply')
    output_path = os.path.join(dec_path, output_filename)
    
    try:
        simplified_mesh.save(output_path)
        save_time = time.time()
        print(f"Saved {output_path} in {save_time - decimate_time:.2f} seconds")
    except Exception as e:
        print(f"Error saving {output_path}: {e}")
    
    total_time = time.time() - start_time
    print(f"Total time: {total_time // 60} minutes and {total_time % 60:.2f} seconds")

# Process each cellid in the astro_list
for cellid in astro_list:
    process_obj(cellid, dec_prcnt, obj_path, dec_path)


Loading data/c3_cell_meshes/5101516913_0.h5
Loaded data/c3_cell_meshes/5101516913_0.h5 in 4.13 seconds
Original mesh info: PolyData (0x14a0b176be8)
  N Cells:	33441811
  N Points:	16707097
  N Strips:	0
  X Bounds:	1.946e+06, 2.039e+06
  Y Bounds:	1.480e+06, 1.607e+06
  Z Bounds:	4.290e+02, 8.795e+04
  N Arrays:	0

Decimating data/c3_cell_meshes/5101516913_0.h5 by 95%
Decimated data/c3_cell_meshes/5101516913_0.h5 in 22271.14 seconds
Simplified mesh info: PolyData (0x14a0b176f48)
  N Cells:	1672090
  N Points:	844717
  N Strips:	0
  X Bounds:	1.946e+06, 2.039e+06
  Y Bounds:	1.481e+06, 1.606e+06
  Z Bounds:	4.290e+02, 8.791e+04
  N Arrays:	0

Saved data/c3_cell_meshes/dec/5101516913_0.ply in 1.72 seconds
Total time: 371.0 minutes and 17.00 seconds
